IMPORT AND SETTINGS

In [195]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import pandas as pd
import cv2 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import random

try:
    tf.config.experimental.enable_op_determinism()
    print("✅ Op Determinism Abilitato!")
except AttributeError:
    print("⚠️ Attenzione: La tua versione di TF è troppo vecchia per enable_op_determinism.")

def reset_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
reset_seeds(42)

2.21.0-dev20251210
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Op Determinism Abilitato!


DATASET LOADING AND MODELLING

In [196]:
def load_dataset():
    folder='dataset/images'
    data=[]
    for filename in sorted(os.listdir(folder)):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path) #opencv save in bgr
        data.append({
            'image':img,
            'filename':filename
        })

    print(len(data),'images loaded')
    print('file name is: ',data[0]['filename'], 'shape of the image is:  ', data[0]['image'].shape )

    label=pd.read_csv('dataset/raw/bbx_annotations.csv')
    print(label.shape, label.iloc[0]['filename'])
    #images order is random, and for 1 image you can have more class

    print('we have', len(label['class'].unique()), 'different classes')

    #replace biggger img with half sized ones
    cv2.imwrite('resize_image/last_img_pre_downsampling.jpg',data[-100]['image'])
    for i,item in enumerate(data):
        if "upper" in item["filename"].lower():
            data[i]['image']=cv2.resize(
                data[i]['image'],
                (data[i]['image'].shape[1]//2,data[i]['image'].shape[0]//2)
                ,interpolation=cv2.INTER_AREA
            )
    cv2.imwrite('resize_image/last_img_post_downsampling.jpg',data[-100]['image'])
       
    return data, label

FROM THE PURE DATASET TO THE TRAIN AND TEST DATA AND LABEL

In [197]:
def dataset_modelling(dataset,annotation):   
    dataset_df = pd.DataFrame(dataset) 
    label_map={'goalpost':0,
               'ball':1,
               'robot':2,
               'goalspot':3,
               'centerspot':4}
    def get_vector(classes_found):
        vec=np.zeros(5,dtype=int)

        for c in classes_found:
            if c in label_map:
                vec[label_map[c]]=1
        return list(vec)
    
    grouped = annotation.groupby('filename')['class'].apply(list).reset_index()
    grouped['label']=grouped['class'].apply(get_vector)
    final_annotation=grouped[['filename','label']]

    final_dataset= pd.merge(dataset_df, final_annotation[['filename', 'label']], on='filename', how='inner')
    final_dataset.to_csv('csv/temp/final_dataset.csv')
    final_dataset=final_dataset.drop(columns=['filename'])
    df_train, df_test = train_test_split(final_dataset, test_size=0.2, random_state=42)
    x_train = np.array(df_train['image'].tolist()).astype('float32') / 255.0
    y_train = np.array(df_train['label'].tolist()).astype('float32')
    
    x_test = np.array(df_test['image'].tolist()).astype('float32') / 255.0
    y_test = np.array(df_test['label'].tolist()).astype('float32')
    return x_train, y_train,x_test,y_test
    

COMPUTING THE WEIGHTS FOR EACH CLASS, TO GIVE MORE IMPORTANCE TO UNDER-REPRESENTED CLASS

In [198]:
def compute_class_weight(y_train):
    counts = np.sum(y_train, axis=0)
    tot_samples=np.sum(counts)
    num_class=5
    class_weight_dict={}

    for i,count in enumerate(counts):
        weight=tot_samples/(num_class*count)
        class_weight_dict[i]=weight

        
    return class_weight_dict


CREATE ALL DIFFERENT COMBINATION OF HYPERPARAMETER, WITHOUT USING GRIDSEARCH

In [199]:
def create_hyperparam_combination():
    param_grid = {
    'batch_size': [64,16],
    'layer_number':[4,5],
    'kernel_dim': [3,7],
    'pool_dim': [2,3], 
    'lr': [0.0001,0.001],
    'fc1' : [128],
    'fc2': [128]     
}

    #every possible combination
    keys, values = zip(*param_grid.items())
    combinations = list(itertools.product(*values))
    combinations_dicts = [dict(zip(keys, v)) for v in combinations]
    return combinations_dicts

MODEL BUILIDNG, 
kernel dimesnsion, pooling dimension, fc layers dimension, number of conv layer and learning rate have different combination.
instead, i fixed:
pooling stride=2 
pooling type: avg pooling
number of kernel per layer: 16, 32, 64...
last pooling: glob avg pool

In [200]:
def build_model(layer_num,kernel_dim,pool_dim,fc1,fc2):
    model=models.Sequential()
    model.add(tf.keras.Input(shape=(240,320,3)))

    for i in range(layer_num):
        kernel_number=16*(2**i)
        model.add(layers.Conv2D(kernel_number,(kernel_dim,kernel_dim),activation='relu',padding='same'))
        model.add(layers.AveragePooling2D((pool_dim,pool_dim),strides=2,padding='same'))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(fc1,activation='relu'))
    model.add(layers.Dense(fc2,activation='relu'))
    model.add(layers.Dense(5,activation='sigmoid'))

    #model.summary()
    return model


SAVING THE CSV FILE CONTAINING F1S OF EACH SEARCHED MODEL

In [201]:
def saving_csv(report_dict,param,target_names,all_results):
    current_result = param.copy()    
    for class_name in target_names:
        current_result[f'f1_{class_name}'] = report_dict[class_name]['f1-score']
    current_result['f1_macro_avg'] = report_dict['macro avg']['f1-score']

    all_results.append(current_result)
    pd.DataFrame(all_results).to_csv('csv/f1_search/search3.csv', index=False)



MAIN

In [ ]:
dataset, annotation=load_dataset()
x_train,y_train, x_test, y_test=dataset_modelling(dataset, annotation)
combination=create_hyperparam_combination()

all_results = []
bestf1=0
for i,param in enumerate(combination):

    
    print(f"\nTRAINING RUN {i+1}/{len(combination)}")
    tf.keras.backend.clear_session()
    reset_seeds(42)
    model=build_model( param['layer_number'],param['kernel_dim'],param['pool_dim'],param['fc1'],param['fc2'])
    opt=tf.keras.optimizers.Adam(learning_rate=param['lr'])
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall'),
                          ])
    early_stop=EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
    )
    reset_seeds(42)
    class_weights_dict=compute_class_weight(y_train)
    history=model.fit(
        x_train,y_train,
        epochs=120,
        batch_size=param['batch_size'], 
        validation_split=0.2, 
        callbacks=[early_stop],
        class_weight=class_weights_dict)

    y_pred=model.predict(x_test)
    predictions_binary = (y_pred > 0.5).astype(int)
    target_names = ['goalpost','ball','robot','goalspot','centerspot']
    report_dict = classification_report(y_test, predictions_binary, target_names=target_names, output_dict=True)
    f1_macro = report_dict['macro avg']['f1-score']
    if f1_macro>bestf1:
        best_report_dict=report_dict
        best_param=param
        bestf1=f1_macro
    saving_csv(report_dict,param,target_names,all_results)


df_report = pd.DataFrame(best_report_dict).transpose()
df_report = df_report.round(2)
df_report['support'] = df_report['support'].astype(int)
csv_path = 'csv/report/report_bestcomb3.csv'
df_report.to_csv(csv_path)
print(best_param)




2452 images loaded
file name is:  lower_100056_jpg.rf.ec9852c66b4eee4a185317210a378f16.jpg shape of the image is:   (240, 320, 3)
(8125, 8) upper_604302_jpg.rf.6215ee30a829ec658154eb4d067dfdf5.jpg
we have 5 different classes

TRAINING RUN 1/32


TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'class_weights'